# Buffer-Based Density Calculation
This method involves counting the number of points (buildings) within a 250m neighbourhood radius for each raster cell.
### 1. Create a Fishnet/Grid Over the Study Area

In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box

# Load your point layer
buildings = gpd.read_file("C:/GIS_Course/MScThesis-MaviSantarelli/data/Predictors/UrbanDensity/Buildings_Points.shp")

# Define grid resolution and extent
resolution = 30  # 30m cell size
xmin, ymin, xmax, ymax = buildings.total_bounds  # Get the extent from the building points

# Create the grid cells
rows = int((ymax - ymin) / resolution)
cols = int((xmax - xmin) / resolution)
x_coords = np.arange(xmin, xmax, resolution)
y_coords = np.arange(ymin, ymax, resolution)
grid_cells = [box(x, y, x + resolution, y + resolution) for x in x_coords for y in y_coords]

# Create a GeoDataFrame from the grid cells
grid = gpd.GeoDataFrame(geometry=grid_cells, crs=buildings.crs)


### 2. Buffer Each Grid Cell and Count Points 
Buffer each grid cell by 250m and count the number of points (buildings) within the buffer.

In [3]:
# Reproject both the grid and building points to a projected CRS (e.g., EPSG:27700 - British National Grid)
projected_crs = "EPSG:27700"
buildings = buildings.to_crs(projected_crs)
grid = grid.to_crs(projected_crs)

# Buffer each grid cell by 250m
grid["buffered"] = grid.geometry.buffer(250)

# Spatial join to count points within each buffer
grid["building_count"] = grid["buffered"].apply(
    lambda buf: buildings[buildings.geometry.within(buf)].shape[0]
)

# Remove the buffer geometry to keep only the grid and counts
grid = grid.drop(columns="buffered")

### 3. Normalise the Density 
Normalise the count by the area of the buffer (πr², where r = 250m):

In [4]:
import numpy as np

# Area of a 250m circular buffer in square metres
buffer_area = np.pi * (250**2)

# Normalise the building count by dividing by the buffer area
grid["building_density"] = grid["building_count"] / buffer_area

### 4. Tile Based Rasterisation
The code below performs tile-based rasterisation of a grid containing normalised building density values to create a raster predictor layer for use in species distribution modelling (SDM). The process is divided into smaller tiles (chunks) to avoid memory errors when working with large datasets and fine resolution (30m).



In [ ]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
from shapely.geometry import box

# Load the grid GeoDataFrame with building density
grid = grid  # Assuming grid already has "building_density" column from previous steps

# Define grid resolution and extent
resolution = 30  # 30m cell size
xmin, ymin, xmax, ymax = grid.total_bounds  # Get the extent from the grid
tile_size = 10000  # Tile size in metres (10km x 10km tiles)

# Create a GeoTIFF file to write the raster incrementally
out_shape = (int((ymax - ymin) / resolution), int((xmax - xmin) / resolution))
transform = from_origin(xmin, ymax, resolution, resolution)

with rasterio.open(
    "C:/GIS_Course/MScThesis-MaviSantarelli/data/Predictors/UrbanDensity/urban_density_tiles.tif",
    "w",
    driver="GTiff",
    height=out_shape[0],
    width=out_shape[1],
    count=1,
    dtype="float32",
    crs=grid.crs,
    transform=transform,
) as dst:

    # Loop over tiles in x and y directions
    for x_start in np.arange(xmin, xmax, tile_size):
        for y_start in np.arange(ymin, ymax, tile_size):
            x_end = min(x_start + tile_size, xmax)
            y_end = min(y_start + tile_size, ymax)
            
            # Create a bounding box for the current tile
            tile_bbox = box(x_start, y_start, x_end, y_end)
            
            # Select grid cells that intersect the current tile
            tile_grid = grid[grid.intersects(tile_bbox)]
            
            # Skip empty tiles
            if tile_grid.empty:
                continue
            
            # Rasterize the selected grid cells
            tile_array = rasterize(
                [(geom, value) for geom, value in zip(tile_grid.geometry, tile_grid["building_density"])],
                out_shape=(int((y_end - y_start) / resolution), int((x_end - x_start) / resolution)),
                transform=from_origin(x_start, y_end, resolution, resolution),
                fill=0,
                dtype="float32",
            )
            
            # Calculate the window position in the output raster
            row_start = int((ymax - y_end) / resolution)
            col_start = int((x_start - xmin) / resolution)
            window = rasterio.windows.Window(col_start, row_start, tile_array.shape[1], tile_array.shape[0])
            
            # Write the tile array to the output raster
            dst.write(tile_array, 1, window=window)


### 5. Plot

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Open the generated raster
raster_path = "C:/GIS_Course/MScThesis-MaviSantarelli/data/Predictors/UrbanDensity/urban_density_tiles.tif"
with rasterio.open(raster_path) as src:
    data = src.read(1)  # Read the first band (building density)
    
    # Plot the raster data
    plt.figure(figsize=(10, 8))
    plt.imshow(data, cmap="viridis", extent=src.bounds)
    plt.colorbar(label="Normalised Building Density")
    plt.title("Urban Density (Normalised)")
    plt.xlabel("Longitude (metres)")
    plt.ylabel("Latitude (metres)")
    plt.show()
